In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP
from statsmodels.stats.multitest import multipletests

from matplotlib.colors import LinearSegmentedColormap

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  


In [ ]:
outfigdir = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures/250409"
!mkdir -p $outfigdir

# load gene annotation and data

In [ ]:
gene_modules = GeneModules()
g, gs, ms = gene_modules.check_genes('Cdh13')
print("\t".join(g))
print("\t".join(gs))
print("\t".join(ms))

In [ ]:
genes_alltime_hvgs = np.loadtxt('/u/home/f/f7xiesnm/v1_multiome/l23_alltime_hvgs_n4940.txt', dtype='str')
genes_alltime_hvgs

In [ ]:
# use those 286 genes
df = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/cheng21_cell_scrna/res/L23-ABC-genes-n288-n286unq-annot_v3_july8_2024.csv")
genes_l23 = df['gene'].astype(str).values
genes_l23a = df[df['P17on']=='A']['gene'].astype(str).values
genes_l23b = df[df['P17on']=='B']['gene'].astype(str).values
genes_l23c = df[df['P17on']=='C']['gene'].astype(str).values

print(genes_l23a.shape, genes_l23b.shape, genes_l23c.shape)
genes_grp = df['P17on'].astype(str).values
assert len(genes_l23) == len(np.unique(genes_l23))

genes_l23.shape

In [ ]:
genes_alltime_hvgs_rm_l23 = genes_alltime_hvgs[~np.isin(genes_alltime_hvgs, genes_l23)]
genes_alltime_hvgs_rm_l23.shape

In [ ]:
scores_abc = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/scores_l23abc.csv", 
                         index_col=0,
                        )
scores_abc['scores_c-a'] = scores_abc['scores_c'] - scores_abc['scores_a']
scores_abc

In [ ]:
adata = anndata.read("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad")
adata

In [ ]:
adata.X = adata.raw.X

In [ ]:
adata.obs['scores_a'] = scores_abc.loc[adata.obs.index,'scores_a'].copy()
adata.obs['scores_b'] = scores_abc.loc[adata.obs.index,'scores_b'].copy()
adata.obs['scores_c'] = scores_abc.loc[adata.obs.index,'scores_c'].copy()
adata.obs['scores_c-a'] = scores_abc.loc[adata.obs.index,'scores_c-a'].copy()

In [ ]:
sample_labels = adata.obs['Sample'].values
time_labels = [s[:-1].replace('DR', '') for s in sample_labels]

adata.obs['sample'] = sample_labels #
adata.obs['time']   = time_labels

uniq_samples = natsorted(np.unique(sample_labels))
nr_samples = [s for s in uniq_samples if "DR" not in s]
dr_samples = [s for s in uniq_samples if "DR" in s]

uniq_conds = np.array(natsorted(np.unique(adata.obs['cond'].values)))
print(uniq_conds)

In [ ]:
nr_idx = np.array([0,1,2,4,6,8,10])
dr_idx = np.array([3,5,7,9])

nr_times = np.array([6,8,10,12,14,17,21])
dr_times = np.array(       [12,14,17,21])

In [ ]:
# remove mitocondria genes
adata = adata[:,~adata.var.index.str.contains(r'^mt-')]
# remove sex genes
adata = adata[:,~adata.var.index.str.contains(r'^Xist$')]

# filter genes
cond = np.ravel((adata.X>0).sum(axis=0)) > 10 # expressed in more than 10 cells
adata = adata[:,cond].copy()

In [ ]:
adata

In [ ]:
# output = '/u/home/f/f7xiesnm/v1_multiome/multiome_l23_allgenes.txt'

# all_genes = adata.var.index.values
# print(all_genes.shape)
# np.savetxt(output, all_genes, fmt='%s')

In [ ]:
# counts
x = adata.X
cov = np.ravel(np.sum(x, axis=1))
genes = adata.var.index.values

# CP10k
xn = (sparse.diags(1/cov).dot(x))*1e4

# log2(CP10k+1)
xln = xn.copy()
xln.data = np.log2(xln.data+1)

adata.layers[    'norm'] = np.array(xn.todense())
adata.layers[ 'lognorm'] = np.array(xln.todense())

In [ ]:
genes_idx_alltime_hvgs_rm_l23 = basicu.get_index_from_array(adata.var.index.values, genes_alltime_hvgs_rm_l23)
genes_idx_alltime_hvgs_rm_l23

In [ ]:
np.random.rand(0)

num_archetypal_cells = 100
offset = 1 # CP10k + offset (CPM + 100*offset)
SHUFFLE = False #False

n_pseudo_genes = 35

n_cond = len(uniq_conds)
n_gene = adata.shape[1] 

qs_tensor   = np.zeros((n_cond,3,n_gene))  # 3 represents 3 pairwise comparisons (ca, ba, bc)
l2fc_tensor = np.zeros((n_cond,3,n_gene))


for cond_code, cond in enumerate(uniq_conds):
    # get sub
    adatasub = adata[adata.obs['cond']==cond]
    n_cells = adatasub.shape[0]
    
    # get A vs C 
    ranks_ac = adatasub.obs['scores_c-a'].rank()
    ranks_b  = adatasub.obs['scores_b'].rank()
    
    precond_a = ranks_ac <= num_archetypal_cells
    precond_c = ranks_ac > adatasub.shape[0] - num_archetypal_cells
    precond_b = ranks_b  > adatasub.shape[0] - num_archetypal_cells
    
    cond_a = np.all([ precond_a, ~precond_b, ~precond_c], axis=0)
    cond_b = np.all([~precond_a,  precond_b, ~precond_c], axis=0)
    cond_c = np.all([~precond_a, ~precond_b,  precond_c], axis=0)
    
    # SHUFFLE
    if SHUFFLE:
        adatasub = adatasub[np.random.choice(n_cells, size=n_cells, replace=False)]
    
    # print(precond_a.sum(), 
    #       precond_b.sum(), 
    #       precond_c.sum(),)
    print(cond, cond_a.sum(), cond_b.sum(), cond_c.sum())
    
    adatasub_a = adatasub[cond_a]
    adatasub_b = adatasub[cond_b]
    adatasub_c = adatasub[cond_c]
    
    # DEGs
    mat_a = adatasub_a.layers['norm'][...]
    mat_b = adatasub_b.layers['norm'][...]
    mat_c = adatasub_c.layers['norm'][...]
    
    logmat_a = adatasub_a.layers['lognorm'][...]
    logmat_b = adatasub_b.layers['lognorm'][...]
    logmat_c = adatasub_c.layers['lognorm'][...]
    
    ts_ca, ps_ca = stats.ttest_ind(logmat_c, logmat_a)
    ts_ba, ps_ba = stats.ttest_ind(logmat_b, logmat_a)
    ts_bc, ps_bc = stats.ttest_ind(logmat_b, logmat_c)
    
    _, qs_ca, _, _ = multipletests(np.nan_to_num(ps_ca, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    _, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    _, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    
    l2fc_ca = np.log2(np.mean(mat_c, axis=0)+offset) - np.log2(np.mean(mat_a, axis=0)+offset) # log2FC (CP10k as raw counts)
    l2fc_ba = np.log2(np.mean(mat_b, axis=0)+offset) - np.log2(np.mean(mat_a, axis=0)+offset) # log2FC (CP10k as raw counts)
    l2fc_bc = np.log2(np.mean(mat_b, axis=0)+offset) - np.log2(np.mean(mat_c, axis=0)+offset) # log2FC (CP10k as raw counts)
    
    qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
    qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
    qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

    l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
    l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
    l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change

    cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > 1, qs_a < 0.05], axis=0)
    cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > 1, qs_c < 0.05], axis=0)
    cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > 1, qs_b < 0.05], axis=0)
    
    # save this
    l2fc_tensor[cond_code, 0] = l2fc_ca
    l2fc_tensor[cond_code, 1] = l2fc_ba
    l2fc_tensor[cond_code, 2] = l2fc_bc
    
    qs_tensor[cond_code, 0] = qs_ca
    qs_tensor[cond_code, 1] = qs_ba
    qs_tensor[cond_code, 2] = qs_bc
    
    # [0,1] scaled scores
    mat = adatasub.layers['lognorm'][...]
    mins = np.min(mat, axis=0)
    maxs = np.max(mat, axis=0)
    nmat = (mat - mins)/(maxs-mins+1e-10)
    
    # print(cond, cond_sig_a.sum(), cond_sig_c.sum(), cond_sig_b.sum()) 

# output, check results and stats

In [ ]:
# %%time
# fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_qs_250409.npy'
# fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_l2fc_250409.npy'
# fout3 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_gene_list_250409.csv'

# np.save(fout1, qs_tensor)
# np.save(fout2, l2fc_tensor)

In [ ]:
# qs_tensor = np.load(fout1)
# l2fc_tensor = np.load(fout2)

l2fc_th = np.log2(2)
l2fc_th_s = np.log2(1.2)
alpha_th = 0.05

In [ ]:
qs_ca   = qs_tensor[:,0,:]
qs_ba   = qs_tensor[:,1,:]
qs_bc   = qs_tensor[:,2,:]

l2fc_ca = l2fc_tensor[:,0,:]
l2fc_ba = l2fc_tensor[:,1,:]
l2fc_bc = l2fc_tensor[:,2,:]

In [ ]:
qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


l2fc_as = np.min([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_cs = np.min([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_bs = np.min([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th], axis=0)
cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th], axis=0)
# cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th, l2fc_as > l2fc_th_s], axis=0)
# cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th, l2fc_cs > l2fc_th_s], axis=0)
cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th, l2fc_bs > l2fc_th_s], axis=0)


instances, counts_a = np.unique(cond_sig_a.sum(axis=0), return_counts=True)
instances, counts_c = np.unique(cond_sig_c.sum(axis=0), return_counts=True)
instances, counts_b = np.unique(cond_sig_b.sum(axis=0), return_counts=True)

print('num A genes for each cond:\t', cond_sig_a.sum(axis=1), np.any(cond_sig_a, axis=0).sum())
print('num C genes for each cond:\t', cond_sig_c.sum(axis=1), np.any(cond_sig_c, axis=0).sum())
print('num B genes for each cond:\t', cond_sig_b.sum(axis=1), np.any(cond_sig_b, axis=0).sum())

print('num A genes in num conds:\t',  counts_a[1:])
print('num C genes in num conds:\t',  counts_c[1:])
print('num B genes in num conds:\t',  counts_b[1:])

In [ ]:
df_res_all = []

for label, cond_sig in zip(['A', 'C', 'B'], 
                           [cond_sig_a, cond_sig_c, cond_sig_b]):
    
    cond_idx, gene_idx = np.nonzero(cond_sig.astype(int))
    
    df_res = pd.DataFrame()
    df_res['cond'] = uniq_conds[cond_idx]
    df_res['gene'] = genes[gene_idx]
    df_res['archetype'] = label
    df_res_all.append(df_res)
    
df_res_all = pd.concat(df_res_all)
df_res_all
    

In [ ]:
df_res_all.groupby('gene').sum()

In [ ]:
# df_res_all.to_csv(fout3, header=True, index=False)

# further check

In [ ]:
a_any = np.sort(adata.var[np.any(cond_sig_a, axis=0)].index.values)
a_all = np.sort(adata.var[np.all(cond_sig_a, axis=0)].index.values)

c_any = np.sort(adata.var[np.any(cond_sig_c, axis=0)].index.values)
c_all = np.sort(adata.var[np.all(cond_sig_c, axis=0)].index.values)
ac_overlap = np.sort(adata.var[np.logical_and(np.any(cond_sig_a, axis=0), np.any(cond_sig_c, axis=0))].index.values)

b_any = np.sort(adata.var[np.any(cond_sig_b, axis=0)].index.values)
b_all = np.sort(adata.var[np.all(cond_sig_b, axis=0)].index.values)

print('a any', a_any.shape)
print('a all', a_all.shape)

print('c any', c_any.shape)
print('c all', c_all.shape)
print('ac overlap', ac_overlap.shape)

print('b any', b_any.shape)
print('b all', b_all.shape)

In [ ]:
a_all_annots, a_all_styled, a_all_annots_styled = gene_modules.check_genes(a_all)
c_all_annots, c_all_styled, c_all_annots_styled = gene_modules.check_genes(c_all)
b_all_annots, b_all_styled, b_all_annots_styled = gene_modules.check_genes(b_all)

ac_overlap_annots, ac_overlap_styled, ac_overlap_annots_styled = gene_modules.check_genes(ac_overlap)

print("\t".join(a_all_annots_styled)) # _styled))
print("---"*10) # _styled))
print("\t".join(c_all_annots_styled)) # _styled))
print("---"*10) # _styled))
print("\t".join(b_all_annots_styled)) # _styled))
print("---"*10) # _styled))
print("\t".join(ac_overlap_annots_styled)) # _styled))

In [ ]:
nums_a = cond_sig_a.sum(axis=1)
nums_b = cond_sig_b.sum(axis=1)
nums_c = cond_sig_c.sum(axis=1)

fig, axs = plt.subplots(1,3,figsize=(3*3,4))
ax = axs[0]
ax.plot(nr_times, nums_a[nr_idx], '-o' , fillstyle='none', label='A', color='C0')
ax.set_xticks([6,10,14,17,21])
ax.grid(False, axis='x')
ax.set_ylim(ymin=0, ymax=110)
# ax.legend()
ax.set_ylabel('num. of gene')
ax.set_xlabel('time (P)')
ax.set_title('A genes')
sns.despine(ax=ax)

ax = axs[1]
ax.plot(nr_times, nums_c[nr_idx], '-o' , fillstyle='none', label='C', color='C2')
ax.set_xticks([6,10,14,17,21])
ax.grid(False, axis='x')
ax.set_ylim(ymin=0, ymax=110)
ax.set_title('C genes')
# ax.legend()
sns.despine(ax=ax)

ax = axs[2]
ax.plot(nr_times, nums_b[nr_idx], '-o' , fillstyle='none', label='B', color='C1')
ax.set_xticks([6,10,14,17,21])
ax.grid(False, axis='x')
ax.set_ylim(ymin=0, ymax=70)
# ax.legend()
sns.despine(ax=ax)
ax.set_title('B genes')
fig.tight_layout()
output = os.path.join(outfigdir, 'num_degs_abc_2.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
num_uniq_a = np.sum(np.any(cond_sig_a, axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c, axis=0))

# check effect size 

In [ ]:
cond_sig_a_any = np.any(cond_sig_a, axis=0)
cond_sig_b_any = np.any(cond_sig_b, axis=0)
cond_sig_c_any = np.any(cond_sig_c, axis=0)
cond_sig_abc_any = np.any(cond_sig_a+cond_sig_b+cond_sig_c, axis=0)
cond_sig_a_any.sum(), cond_sig_b_any.sum(), cond_sig_c_any.sum(), cond_sig_abc_any.sum()

In [ ]:
# adata.obs['sample'].unique()
import re

todo_conds = [
    'P12DR', 'P14DR', 'P17DR', 'P21DR',
    'P6', 'P8', 'P10', 'P12', 'P14', 'P17', 'P21', 
]
todo_samps = [
    'P12DRa', 'P12DRb',
    'P14DRa', 'P14DRb',
    'P17DRa', 'P17DRb',
    'P21DRa', 'P21DRb',
    'P6a', 'P6b', 'P6c', 
    'P8a', 'P8b', 'P8c', 
    'P10a', 'P10b', 
    'P12a', 'P12b', 'P12c', 
    'P14a', 'P14b',
    'P17a', 'P17b', 
    'P21a', 'P21b', 
]
todo_conds_t = np.array([int(re.sub(r'[a-zA-Z]', '', a)) for a in todo_conds])
todo_samps_t = np.array([int(re.sub(r'[a-zA-Z]', '', a)) for a in todo_samps])
print(todo_conds_t)
print(todo_samps_t)

def mean_over_samples(mmat_res_samp):
    """25 samples to 11 conditions
    """
    assert mmat_res_samp.shape[0] == 25
    
    mmat_res_samp_mean = np.zeros(mmat_res_samp.shape)[:11]
    mmat_res_samp_mean[0] = np.mean(mmat_res_samp[ :2], axis=0)
    mmat_res_samp_mean[1] = np.mean(mmat_res_samp[2:4], axis=0)
    mmat_res_samp_mean[2] = np.mean(mmat_res_samp[4:6], axis=0)
    mmat_res_samp_mean[3] = np.mean(mmat_res_samp[6:8], axis=0)

    mmat_res_samp_mean[4] = np.mean(mmat_res_samp[8:11], axis=0)
    mmat_res_samp_mean[5] = np.mean(mmat_res_samp[11:14], axis=0)
    mmat_res_samp_mean[6] = np.mean(mmat_res_samp[14:16], axis=0)
    mmat_res_samp_mean[7] = np.mean(mmat_res_samp[16:19], axis=0)
    mmat_res_samp_mean[8] = np.mean(mmat_res_samp[19:21], axis=0)
    mmat_res_samp_mean[9] = np.mean(mmat_res_samp[21:23], axis=0)
    mmat_res_samp_mean[10] = np.mean(mmat_res_samp[23:  ], axis=0)
    
    return mmat_res_samp_mean

def transform_bigredmat(bigmat, n_type):
    """bigmat or redmat
    to fmat and zmat
    """
    fmat = bigmat.reshape(-1, bigmat.shape[-1]).T
    fmat = np.hstack([fmat[:,4*n_type:], fmat[:,:4*n_type]]) # CHANGED COLUMN ORDER!!
    zmat = zscore(fmat, axis=1)
    
    return fmat, zmat

In [ ]:
%%time

mat = adata.layers['norm'][...]
gexp_l23baseline = np.log2(np.mean(mat, axis=0)*1e2+offset) # CP10k -> CPM

n_type = 10
frac_archetypal_cells_viz = 0.2
bigmat_nfd = np.zeros((len(todo_samps), n_type, mat.shape[1]))
bigmat_abc = np.zeros((len(todo_samps),      3, mat.shape[1]))

for i, samp in enumerate(todo_samps):
    print(samp)
    
    # get sub
    adatasub = adata[adata.obs['sample']==samp]
    n_cells = adatasub.shape[0]
    
    # get A vs C 
    ranks_ac = adatasub.obs['scores_c-a'].rank()
    ranks_b  = adatasub.obs['scores_b'].rank()
    
    # per type
    cells_type_nfd = pd.qcut(ranks_ac, n_type, labels=False)
    for j in range(n_type):
        mat_j = adatasub[cells_type_nfd==j].layers['norm'][...]
        mmat_j = np.log2(np.mean(mat_j, axis=0)*1e2+offset)-gexp_l23baseline # CP10k -> CPM
        bigmat_nfd[i,j] = mmat_j
    
    # A, B, C
    num_archetypal_cells_viz = int(n_cells*frac_archetypal_cells_viz)
    
    precond_a = ranks_ac <= num_archetypal_cells_viz
    precond_c = ranks_ac > adatasub.shape[0] - num_archetypal_cells_viz
    precond_b = ranks_b  > adatasub.shape[0] - num_archetypal_cells_viz
    
    cond_a = np.all([ precond_a, ~precond_b, ~precond_c], axis=0)
    cond_b = np.all([~precond_a,  precond_b, ~precond_c], axis=0)
    cond_c = np.all([~precond_a, ~precond_b,  precond_c], axis=0)
    
    for j, cond in enumerate([cond_a, cond_b, cond_c]):
        mat_j = adatasub[cond].layers['norm'][...]
        mmat_j = np.log2(np.mean(mat_j, axis=0)*1e2+offset)-gexp_l23baseline # CP10k -> CPM
        bigmat_abc[i,j] = mmat_j


In [ ]:
bigmat_abc_ig_list = [
    np.mean(bigmat_abc[:,:,cond_sig_a_any], axis=-1),
    np.mean(bigmat_abc[:,:,cond_sig_b_any], axis=-1),
    np.mean(bigmat_abc[:,:,cond_sig_c_any], axis=-1),
]

redmat_abc_ig_list = [mean_over_samples(x) for x in bigmat_abc_ig_list]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*3,1*3), sharex=True, sharey=True)
for i in range(3):
    ax = axs[i]
    bigmat_mean_ig = bigmat_abc_ig_list[i]
    redmat_mean_ig = redmat_abc_ig_list[i]
    
    ax.plot(todo_samps_t[8:], bigmat_mean_ig[8:,0], 'o', markersize=5, fillstyle='none', color='C0')
    ax.plot(todo_samps_t[8:], bigmat_mean_ig[8:,1], 'o', markersize=5, fillstyle='none', color='C1')
    ax.plot(todo_samps_t[8:], bigmat_mean_ig[8:,2], 'o', markersize=5, fillstyle='none', color='C2')
    
    ax.plot(todo_samps_t[:8], bigmat_mean_ig[:8,0], 's', markersize=5, fillstyle='none', color='C0', alpha=0.5)
    ax.plot(todo_samps_t[:8], bigmat_mean_ig[:8,1], 's', markersize=5, fillstyle='none', color='C1', alpha=0.5)
    ax.plot(todo_samps_t[:8], bigmat_mean_ig[:8,2], 's', markersize=5, fillstyle='none', color='C2', alpha=0.5)
    
    ax.plot(todo_conds_t[4:], redmat_mean_ig[4:,0], '-', color='C0')
    ax.plot(todo_conds_t[4:], redmat_mean_ig[4:,1], '-', color='C1')
    ax.plot(todo_conds_t[4:], redmat_mean_ig[4:,2], '-', color='C2')
    
    ax.plot(todo_conds_t[:4], redmat_mean_ig[:4,0], '-', color='C0', alpha=0.5)
    ax.plot(todo_conds_t[:4], redmat_mean_ig[:4,1], '-', color='C1', alpha=0.5)
    ax.plot(todo_conds_t[:4], redmat_mean_ig[:4,2], '-', color='C2', alpha=0.5)

    ax.grid(False)
    ax.set_xticks([6,10,14,17,21])
    sns.despine(ax=ax)

axs[0].set_xlabel('Postnatal day (P)')
axs[0].set_ylabel('Gene expr.\nlog2(archetype / baseline)')
axs[0].set_title(f'A genes\nn={num_uniq_a:,}')
axs[1].set_title(f'B genes\nn={num_uniq_b:,}')
axs[2].set_title(f'C genes\nn={num_uniq_c:,}')
output = os.path.join(outfigdir, 'abc_degs_signals_over_time_withDR.pdf') 
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
redmat_nfd = mean_over_samples(bigmat_nfd)
fmat_nfd, zmat_nfd = transform_bigredmat(redmat_nfd, n_type)
print(redmat_nfd.shape) # cond, type, gene
print(fmat_nfd.shape)   # gene, cond*type
print(zmat_nfd.shape)   # gene, cond*type

fmat_nfd_ag = fmat_nfd[cond_sig_a_any]
fmat_nfd_bg = fmat_nfd[cond_sig_b_any]
fmat_nfd_cg = fmat_nfd[cond_sig_c_any]
fmat_nfd_abcg = fmat_nfd[cond_sig_abc_any]

zmat_nfd_ag = zmat_nfd[cond_sig_a_any]
zmat_nfd_bg = zmat_nfd[cond_sig_b_any]
zmat_nfd_cg = zmat_nfd[cond_sig_c_any]
zmat_nfd_abcg = zmat_nfd[cond_sig_abc_any]

print(fmat_nfd_ag.shape, zmat_nfd_ag.shape, zmat_nfd_abcg.shape)

In [ ]:
redmat_nfd_ag = redmat_nfd[:,:,cond_sig_a_any] #.shape
redmat_nfd_bg = redmat_nfd[:,:,cond_sig_b_any] #.shape
redmat_nfd_cg = redmat_nfd[:,:,cond_sig_c_any] #.shape
redmat_nfd_abcg = redmat_nfd[:,:,cond_sig_abc_any] #.shape
print(redmat_nfd_ag.shape)

In [ ]:
from sklearn.cluster import KMeans
def mean_shape(vec):
    """
    """
    loc = np.arange(len(vec))
    
    # vec_n = (vec-np.min(vec))/(np.max(vec)-np.min(vec))
    vec_n = np.clip(vec, 0, None)
    vec_n = vec_n/np.sum(vec_n)
    
    ctrd = loc.dot(vec_n)
    return ctrd

def organize_zmat(zmat, fmat, redmat, title='', n_geneset_clsts=5, genes=None):
    """NOTE THAT THE ORDER OF COND is DIFFERRENT BETWEEN (zmat, fmat) - DR first) and (redmat) - NR first)
    """
    method = KMeans(n_clusters=n_geneset_clsts, n_init=10, random_state=0)
    geneset_clst = method.fit_predict(zmat)

    # average over genes per geneset and cell clusters - leave genesets and conditions there
    time_sketches = []
    for i in range(n_geneset_clsts):
        time_sketch = np.mean(redmat[:,:,geneset_clst==i], axis=2) # mean over genes
        time_sketch = np.max(time_sketch, axis=1) # max over cell types
        time_sketches.append(time_sketch)
    time_sketches = np.vstack(time_sketches)[:,4:] # n_geneset_clsts, n_cond (select NR only)

    # clst_order = [2,1,3,4,0]
    # clst_order = np.argsort(np.argmax(ctrds, axis=1)) 
    clst_order = np.argsort([mean_shape(time_sketch) for time_sketch in time_sketches]) 
    geneset_clst_renamed = pd.Series({clst: i for i, clst in enumerate(clst_order)}).reindex(geneset_clst).values
    geneset_order = np.argsort(geneset_clst_renamed)
    
    # reorder 
    genes_ordered = genes[geneset_order]
    clsts_ordered = geneset_clst_renamed[geneset_order]
    zmat_ordered = zmat[geneset_order] 
    fmat_ordered = fmat[geneset_order] 
    
    # gene list per group
    geneset_list = []
    for i in range(n_geneset_clsts):
        geneset_list.append(genes_ordered[clsts_ordered==i])
    
    res = {
        'title': title,
        'order': geneset_order,
        'zmat':  zmat_ordered,
        'fmat':  fmat_ordered,
        'genes': genes_ordered,
        'clst':  clsts_ordered,
        'time_sketches':  time_sketches[clst_order],
        'geneset_list': geneset_list,
    }
    return res

In [ ]:
# ctrds = organize_zmat(zmat_nfd_ag, fmat_nfd_ag, redmat_nfd_ag, title='A genes', genes=genes[cond_sig_a_any])

res_a = organize_zmat(zmat_nfd_ag, fmat_nfd_ag, redmat_nfd_ag, title='A genes', genes=genes[cond_sig_a_any])
res_b = organize_zmat(zmat_nfd_bg, fmat_nfd_bg, redmat_nfd_bg, title='B genes', genes=genes[cond_sig_b_any])
res_c = organize_zmat(zmat_nfd_cg, fmat_nfd_cg, redmat_nfd_cg, title='C genes', genes=genes[cond_sig_c_any])

res_abc = organize_zmat(zmat_nfd_abcg, fmat_nfd_abcg, redmat_nfd_abcg, title='ABC genes', genes=genes[cond_sig_abc_any])

# Profile these modules

# Quantify time vs DR effect
- late are DR sensitive
- (P21NR-P10NR) vs (P21NR vs DR)
- refine this as the average time effect vs average DR effect

In [ ]:
times = np.array([6,8,10,12,14,17,21])
dr_times = np.array([12,14,17,21])

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

colors_a = [(0.0, 'black'), (1.0, 'C0')]      
colors_b = [(0.0, 'black'), (1.0, 'C1')]      
colors_c = [(0.0, 'black'), (1.0, 'C2')]      

# Create a custom colormap using LinearSegmentedColormap
cmap_a = LinearSegmentedColormap.from_list('cmap_a', colors_a)
cmap_b = LinearSegmentedColormap.from_list('cmap_b', colors_b)
cmap_c = LinearSegmentedColormap.from_list('cmap_c', colors_c)

colors_l23 = [
    np.array(cmap_a(1.0)),
    0.7*np.array(cmap_a(1.0))+0.3*np.array(cmap_b(1.0)),
    np.array(cmap_b(1.0)),
    0.7*np.array(cmap_b(1.0))+0.3*np.array(cmap_c(1.0)),
    np.array(cmap_c(1.0)),
]

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
def calc_del(red_y):
    del_t = np.mean(red_y[7+3]-red_y[3+3], axis=0) # mean over ABC
    del_v21 = np.mean(-red_y[7+3]+red_y[3], axis=0) # mean over ABC
    del_v17 = np.mean(-red_y[7+2]+red_y[2], axis=0)
    del_v14 = np.mean(-red_y[7+1]+red_y[1], axis=0)
    del_v12 = np.mean(-red_y[7+0]+red_y[0], axis=0)

    del_varr = np.array([del_v21, del_v17, del_v14, del_v12])
    del_v = np.mean(del_varr, axis=0) # mean over time
    del_v1721 = np.mean(del_varr[:2], axis=0) # mean over time
    
    return del_t, del_v, del_v21

def calc_del_typespec(red_y):
    del_t   = red_y[7+3]-red_y[3+3] # (n_type, n_gene)
    del_v21 = -red_y[7+3]+red_y[3]  #
    del_v17 = -red_y[7+2]+red_y[2]
    del_v14 = -red_y[7+1]+red_y[1]
    del_v12 = -red_y[7+0]+red_y[0]

    del_varr = np.array([del_v21, del_v17, del_v14, del_v12])
    del_v = np.mean(del_varr, axis=0) # mean over time
    del_v1721 = np.mean(del_varr[:2], axis=0) # mean over time
    
    return del_t, del_v, del_v21

In [ ]:
cond_sig_abc_any = np.any([
    cond_sig_a_any, 
    cond_sig_b_any, 
    cond_sig_c_any, 
], axis=0)
print(cond_sig_abc_any.sum())

genes_abc = genes[cond_sig_abc_any]

big_y = bigmat_abc[:,:,cond_sig_abc_any]# .shape
red_y = mean_over_samples(big_y)
del_t_abc, del_v_abc, del_v21_abc = calc_del(red_y)
del_t_abc_typespec, del_v_abc_typespec, del_v21_abc_typespec = calc_del_typespec(red_y)


In [ ]:
big_y = bigmat_nfd[:,:,cond_sig_abc_any]# .shape
red_y = mean_over_samples(big_y)
del_t_nfd, del_v_nfd, del_v21_nfd = calc_del(red_y)
del_t_nfd_typespec, del_v_nfd_typespec, del_v21_nfd_typespec = calc_del_typespec(red_y)
del_t_nfd.shape, del_t_nfd_typespec.shape

In [ ]:
genes_a = genes[cond_sig_a_any]
genes_b = genes[cond_sig_b_any]
genes_c = genes[cond_sig_c_any]

In [ ]:
csm_annots = [
    'igsf',    
    'cad',     
    'fbrn',    
    'eph',     
    'sema',    
    'tene',    
    'astn',    
    'cntnap',  
    'nrxn',    
    'axon',    
    'wiring',
]

In [ ]:
gene_annots_abc = gene_modules.check_genes(genes_abc)[0]
cond_csm_abc = np.array([_g in csm_annots for _g in gene_annots_abc])
print(genes_abc[cond_csm_abc], np.sum(cond_csm_abc))

gene_annots_a = gene_modules.check_genes(genes_a)[0]
cond_csm_a = np.array([_g in csm_annots for _g in gene_annots_a])
print(genes_a[cond_csm_a], np.sum(cond_csm_a))

gene_annots_b = gene_modules.check_genes(genes_b)[0]
cond_csm_b = np.array([_g in csm_annots for _g in gene_annots_b])
print(genes_b[cond_csm_b], np.sum(cond_csm_b))

gene_annots_c = gene_modules.check_genes(genes_c)[0]
cond_csm_c = np.array([_g in csm_annots for _g in gene_annots_c])
print(genes_c[cond_csm_c], np.sum(cond_csm_c))

In [ ]:
selected_conditions = [1,4,6,10]
columns = np.hstack([
    1*n_type+np.arange(n_type), 
    4*n_type+np.arange(n_type), 
    6*n_type+np.arange(n_type), 
   10*n_type+np.arange(n_type),
])

fig, axs = plt.subplots(3,1,figsize=(6,4*3))
for ax_idx, res_this in enumerate([res_a, res_c, res_b]):
    ax = axs[ax_idx]
    
    order = res_this['order']
    title = res_this['title']
    zmat  = res_this['zmat']
    clsts = res_this['clst']
    genes_this = res_this['genes']
    
    sns.heatmap(zmat[:,columns],#[:,:7*10], 
                cmap='coolwarm', cbar_kws=dict(shrink=0.5), 
                xticklabels=False,
                vmax=2.5, vmin=-2.5,
                rasterized=True,
                ax=ax)
    ax.set_yticks(0.5+np.arange(len(zmat)))
    ax.set_yticklabels(genes_this, fontsize=4, rotation=0)
    
    ax.vlines(np.arange(0,7*n_type,n_type), 0, len(zmat), color='white', linewidth=1)
    ax.vlines(7*n_type, 0, len(zmat), color='black', linewidth=1)

    ax.grid(False)
    
ax = axs[0]
# ax.set_xticks(0.5+np.arange(n_type))
# ax.set_xticklabels(['A', '<-', '-', '->', 'C'], fontsize=10, rotation=0)
for i, cond in enumerate(np.hstack([np.array(todo_conds[4:]+todo_conds[:4])[selected_conditions]])):
    # ax.axvline(condcode*5, color='k', linestyle='--', linewidth=1)
    ax.text(i*n_type, -0.5, f'{cond}', fontsize=10, va='bottom')

# output = os.path.join(outfigdir, f'heatmap_csm_all.pdf')
# powerplots.savefig_autodate(fig, output)
plt.show()

# break

# regulon stuff

In [ ]:
f = '/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_trimmed_cleaned_bigtable.csv'
# scenic metadata
df_scenic = pd.read_csv(f, index_col=0)
df_scenic = df_scenic[df_scenic['signs'].isin(["+_+", "-_+"])]

In [ ]:
res = pd.DataFrame()
reg_name_list = [
    # 'Meis2_+_+',
    # 'Tcf12_+_+',
    # 'Fosl2_+_+',
    'Npas4_+_+',
    'Fos_+_+',
    'Sox5_+_+',
    'Cux2_+_+',
    'Egr1_+_+',
    'Satb1_+_+',
    'Smad3_+_+',
    
    # 'Ar_+_+',
    # 'Thra_+_+',
    # 'Nfatc2_+_+',
    # 'Mef2c_+_+',
]
for reg_name in reg_name_list:
    df_this_reg = df_scenic[df_scenic['Consensus_name']==reg_name]
    reg_genes = df_this_reg['Gene'].unique()
    reg_genes_overlap_b = np.intersect1d(reg_genes, res_b['genes'])
    print(reg_name, reg_genes_overlap_b)

In [ ]:
df_scenic[df_scenic['Gene']=='Megf11']['Consensus_name'].unique()

In [ ]:
selected_conditions = [1, 4,8, 6,10]
columns = np.hstack([
    1*n_type+np.arange(n_type), 
    
    4*n_type+np.arange(n_type), 
    8*n_type+np.arange(n_type), 
    
    6*n_type+np.arange(n_type), 
   10*n_type+np.arange(n_type),
])

reg_names = [
    'Npas4_+_+', 
    'Fos_+_+',
    'Fosb_+_+',
    'Fosl2_+_+',
    'Junb_+_+',
    # 'Egr1_+_+',
    # 'Smad3_+_+',
    # 'Jdp2_+_+',
]

fig, ax = plt.subplots(1,1,figsize=(6,3))
res_this = res_b
    
order = res_this['order']
title = res_this['title']
zmat  = res_this['zmat']
clsts = res_this['clst']
genes_this = res_this['genes']

df_this_reg = df_scenic[df_scenic['Consensus_name'].isin(reg_names)]
reg_genes = df_this_reg['Gene'].unique()
reg_genes_overlap = np.intersect1d(reg_genes, genes_this)
print(reg_name, reg_genes_overlap)

cond_this = [_g in reg_genes_overlap for _g in genes_this]
zmat = zmat[cond_this]
clsts = clsts[cond_this]
genes_this = genes_this[cond_this] #res_this['genes']
zmat = zmat[:,columns]

meandiff = np.mean(zmat[:,-n_type:], axis=1)-np.mean(zmat[:,-2*n_type:-n_type], axis=1) # DR-NR
neworder = np.argsort(meandiff)#[::-1]

sns.heatmap(zmat[neworder],#[:,:7*10], 
            cmap='coolwarm', cbar_kws=dict(shrink=0.5), 
            xticklabels=False,
            vmax=2.5, vmin=-2.5,
            rasterized=True,
            ax=ax)
ax.set_yticks(0.5+np.arange(len(zmat)))
ax.set_yticklabels(genes_this[neworder], fontsize=6, rotation=0)

ax.vlines(np.arange(0,7*n_type,n_type), 0, len(zmat), color='white', linewidth=1)
ax.vlines(7*n_type, 0, len(zmat), color='black', linewidth=1)

ax.grid(False)

for i, cond in enumerate(np.hstack([np.array(todo_conds[4:]+todo_conds[:4])[selected_conditions]])):
    ax.text(i*10, -0.5, f'{cond}', fontsize=10, va='bottom')

output = os.path.join(outfigdir, f'heatmap_arg_b_programs.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

# break

# Others

In [ ]:
# selected_conditions = [1,4,6,10]
# columns = np.hstack([
#     1*n_type+np.arange(n_type), 
#     4*n_type+np.arange(n_type), 
#     6*n_type+np.arange(n_type), 
#    10*n_type+np.arange(n_type),
# ])

selected_conditions = [6,10]
columns = np.hstack([
    6*n_type+np.arange(n_type), 
   10*n_type+np.arange(n_type),
])

In [ ]:
vision_abc_csms = [
    # 'Cdh20', 'Cdh7', 'Cdh4', 'Cntnap4', 'Cdh13', 'Igsf9b', 'Megf11', 'Sema6a',
    
    # 'Cntnap5b', 'Cdh20', 'Cdh7', 'Cdh4', 'Pcdh19', 'Pcdh10', 'Epha10',
    #    'Sdk1', 'Cntnap2', 'Cntn6', 'Cntnap4', 'Cdh13', 'Ptprg', 'Igsf9b',
    #    'Sorl1', 'Megf11', 'Flrt2', 'Ghr', 'Cdh18', 'Igsf11', 'Alcam',
    #    'Ncam2', 'Sema6a'
    
    # 'Cntnap5b', 'Cdh20', 'Cdh7', 'Cdh4', 'Cntnap2', 'Cntnap4',
    #    'Igsf9b', 'Megf11', 'Cdh18', 'Igsf11'
    
    
    'Cdh20', 'Cdh7', 'Cdh4', 'Cntnap4', 'Cdh13', 'Igsf9b', 'Megf11',
       'Sema6a'
]


In [ ]:
csms_ordered_record = []

fig, axs = plt.subplots(3,1,figsize=(8,2*3))
for ax_idx, res_this in enumerate([res_a, res_c, res_b]):
    ax = axs[ax_idx]
    
    order = res_this['order']
    title = res_this['title']
    zmat  = res_this['zmat']
    clsts = res_this['clst']
    genes_this = res_this['genes']
    
    # gene_annots_this = gene_modules.check_genes(genes_this)[0]
    cond_csm_this = np.array([_g in vision_abc_csms for _g in genes_this])
    zmat = zmat[cond_csm_this]
    clsts = clsts[cond_csm_this]
    genes_this = genes_this[cond_csm_this] #res_this['genes']

    csms_ordered_record.append(genes_this)
    
    zmat = zmat[:,columns]
    zmat = stats.zscore(zmat, axis=1)
    
    sns.heatmap(zmat,#[:,:7*10], 
                cmap='coolwarm', cbar_kws=dict(shrink=0.5), 
                xticklabels=False,
                vmax=2.5, vmin=-2.5,
                rasterized=True,
                ax=ax)
    ax.set_yticks(0.5+np.arange(len(zmat)))
    ax.set_yticklabels(genes_this, fontsize=12, rotation=0)
    
    # ax.hlines(np.cumsum(np.unique(clsts, return_counts=True)[1]), 0, 11*10, color='white', linewidth=1)
    ax.vlines(np.arange(0,7*10,10), 0, len(zmat), color='white', linewidth=1)
    ax.vlines(7*10, 0, len(zmat), color='black', linewidth=1)

    ax.grid(False)
    
ax = axs[0]
# ax.set_xticks(0.5+np.arange(n_type))
# ax.set_xticklabels(['A', '<-', '-', '->', 'C'], fontsize=10, rotation=0)
for i, cond in enumerate(np.hstack([np.array(todo_conds[4:]+todo_conds[:4])[selected_conditions]])):
    # ax.axvline(condcode*5, color='k', linestyle='--', linewidth=1)
    ax.text(i*10, -0.5, f'{cond}', fontsize=12, va='bottom')

# output = os.path.join(outfigdir, f'heatmap_csm_all.pdf')
# powerplots.savefig_autodate(fig, output)
plt.show()

# break

In [ ]:
csms_ordered_record

In [ ]:
for l in csms_ordered_record:
    assert len(np.unique(l)) == len(l)

In [ ]:
np.intersect1d(
    csms_ordered_record[0],
    csms_ordered_record[1],
)

In [ ]:
np.intersect1d(
    csms_ordered_record[0],
    csms_ordered_record[2],
)

In [ ]:
np.intersect1d(
    csms_ordered_record[1],
    csms_ordered_record[2],
)

In [ ]:
columns = np.hstack([60+np.arange(10), 100+np.arange(10)])
columns

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(8,4*3))
for ax_idx, res_this in enumerate([res_a, res_c, res_b]):
    ax = axs[ax_idx]
    
    order = res_this['order']
    title = res_this['title']
    zmat  = res_this['zmat']
    clsts = res_this['clst']
    genes_this = res_this['genes']
    
    gene_annots_this = gene_modules.check_genes(genes_this)[0]
    cond_csm_this = np.array([_g in csm_annots for _g in gene_annots_this])
    zmat = zmat[cond_csm_this]
    clsts = clsts[cond_csm_this]
    genes_this = genes_this[cond_csm_this] #res_this['genes']

    sns.heatmap(zmat[:,columns],#[:,:7*10], 
                cmap='coolwarm', cbar_kws=dict(shrink=0.5), 
                xticklabels=False,
                vmax=2.5, vmin=-2.5,
                rasterized=True,
                ax=ax)
    ax.set_yticks(0.5+np.arange(len(zmat)))
    ax.set_yticklabels(genes_this, fontsize=4, rotation=0)
    
    # ax.hlines(np.cumsum(np.unique(clsts, return_counts=True)[1]), 0, 11*10, color='white', linewidth=1)
    ax.vlines(np.arange(0,7*10,10), 0, len(zmat), color='white', linewidth=1)
    ax.vlines(7*10, 0, len(zmat), color='black', linewidth=1)

    ax.grid(False)
    
ax = axs[0]
# ax.set_xticks(0.5+np.arange(n_type))
# ax.set_xticklabels(['A', '<-', '-', '->', 'C'], fontsize=10, rotation=0)
for i, cond in enumerate(np.hstack([np.array(todo_conds[4:]+todo_conds[:4])[[6, 10]]])):
    # ax.axvline(condcode*5, color='k', linestyle='--', linewidth=1)
    ax.text(i*10, -0.5, f'{cond}', fontsize=10, va='bottom')

# output = os.path.join(outfigdir, f'heatmap_csm_all.pdf')
# powerplots.savefig_autodate(fig, output)
plt.show()

# break